In [43]:
%run preprocecssing_helpers.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
# %matplotlib inline


In [44]:
path = 'E:\StroopEEG'
eeglab_files = os.listdir(path)

In [45]:
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files

['E:\\StroopEEG\\SA20131216\\20131216_1310.set',
 'E:\\StroopEEG\\SA20131216\\20131216_1441.set',
 'E:\\StroopEEG\\SA20140304\\20140304_1418.set',
 'E:\\StroopEEG\\SA20140304\\20140304_1614.set',
 'E:\\StroopEEG\\SA20140306\\20140306_1133.set',
 'E:\\StroopEEG\\SA20140306\\20140306_1338.set',
 'E:\\StroopEEG\\SA20140313\\20140313_1439.set',
 'E:\\StroopEEG\\SA20140313\\20140313_1630.set',
 'E:\\StroopEEG\\SA20140314\\20140314_1424.set',
 'E:\\StroopEEG\\SA20140314\\20140314_1553.set',
 'E:\\StroopEEG\\SA20140318\\20140318_1043.set',
 'E:\\StroopEEG\\SA20140318\\20140318_1211.set',
 'E:\\StroopEEG\\SA20140319\\20140319_1043.set',
 'E:\\StroopEEG\\SA20140319\\20140319_1342.set',
 'E:\\StroopEEG\\SA20140320A\\20140320_1040.set',
 'E:\\StroopEEG\\SA20140320A\\20140320_1156.set',
 'E:\\StroopEEG\\SA20140320B\\20140320_1421.set',
 'E:\\StroopEEG\\SA20140320B\\20140320_1543.set',
 'E:\\StroopEEG\\SA20140324\\20140324_1441.set',
 'E:\\StroopEEG\\SA20140324\\20140324_1553.set',
 'E:\\StroopEEG\

In [46]:
eeglab_files[0]

'E:\\StroopEEG\\SA20131216\\20131216_1310.set'

In [47]:
# read file
raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)

Reading E:\StroopEEG\SA20131216\20131216_1441.fdt
Reading 0 ... 1406417  =      0.000 ...  2746.908 secs...


In [ ]:
raw.plot(duration)



In [48]:
# clean data
raw.filter(1, None, n_jobs= 10, fir_design= 'firwin')
picks_eeg = mne.pick_types(raw.info, eeg = True, meg = False, eog = False, exclude= 'bads')
method = 'fastica'
n_components= 25
decim = 3
random_state = 23

Setting up high-pass filter at 1 Hz
l_trans_bandwidth chosen to be 1.0 Hz
Filter length of 1691 samples (3.303 sec) selected


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   10.7s
[Parallel(n_jobs=10)]: Done 128 out of 128 | elapsed:   25.9s finished


In [49]:
ica = ICA(n_components= n_components, method = method, random_state= random_state)
print(ica)

<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>


In [50]:
reject = dict(mag = 5e-12, grad = 4000e-13)
ica.fit(raw, decim = decim, reject = reject)

Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selection by number: 25 components
Fitting ICA took 98.3s.


<ICA  |  raw data decomposition, fit (fastica): 468540 samples, 25 components, channels used: "eeg">

In [51]:
ica.detect_artifacts(raw) # can you spot some potential bad guys?

    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    11, 11, 11
    Removing duplicate indices...
Ready.


<ICA  |  raw data decomposition, fit (fastica): 468540 samples, 25 components, channels used: "eeg", 1 sources marked for exclusion>

In [18]:
try: 
    a,b = ica.find_bads_eog(raw)
    ica.exclude += a
except:
    pass

No EOG channels found
Trying with EEG 061 and EEG 062


* Can't plot results due to type mismatching and missing electrode locations.

In [55]:
ica.score_sources

<bound method score_sources of <ICA  |  raw data decomposition, fit (fastica): 468540 samples, 25 components, channels used: "eeg", 1 sources marked for exclusion>>

In [25]:
raw_avg = mne.set_eeg_reference(raw, 'average', projection= False)

Applying average reference.
Applying a custom EEG reference.
